# Leads from Marketing Collateral/Events

This page shows the number of leads (distinct email addresses per month) that access each type of marketing information.  These charts:

* Show the number of leads per month by category
* The same lead can be in more than one category
* A lead is only counted one per month per category
* Each category is on a different scale

## Leads by Category

In [ ]:
import os
pp = '/Users/dane/src/datatools/analytics:/Users/dane/src/datatools'
os.environ.setdefault("PYTHONPATH", pp)

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# standard python includes
import os
import sys
import datetime
from collections import defaultdict, OrderedDict

# my includes
from dtutil.util import print_counters
from dtutil.util import init_cdb
from dtutil.util import Dates

dates = Dates()
month_start = Dates.fmt(dates.month_start)
begin = dates.month_start - datetime.timedelta(days=18*30)
begin = datetime.datetime(begin.year, begin.month, 1)  # get first day of month
begin = Dates.fmt(begin)

# produces:
#    year, month, lead_type, count 
#  where lead_type is one of Event, Training, Webinar, Collateral and TX-LINE
#
query = """
with tmp as (
select distinct email,
    case
        when download like '%TX-LINE%' then 'TX-LINE'
        when download like '%White Paper%' then 'Collateral'
        when download like '%eBook%'       then 'Collateral'
        when download like '%Document%'    then 'Collateral'
        when download like '%Publication%' then 'Collateral'
        when download like '%App Note%'    then 'Collateral'
        when download like '%Newsletter%'  then 'Collateral'
        when download like 'EuMW%'         then 'Collateral'
        when download like '%Workshop%'  then 'Event'
        when download like '%Webinar%'   then 'Webinar'
        when download like '%PA Forum%'  then 'Event'
        when download like '%ADF%'       then 'Event'
        when download like '%Tradeshow%' then 'Event'
        when download like '%Event Reg%' then 'Event'
        when download like '%Party%'     then 'Event'
        when download like '%Training%' then 'Training'
        when download like '%AntSyn%'   then 'AntSyn'
        when download like '%X5%'       then 'AntSyn'
        else download
    end as lead_type,
    datepart(year, entered_date) as year,
    datepart(month, entered_date) as month
  from lds_ip_history with(nolock)
    where entered_date between '{0}' and '{1}'
    and source='registered'
    and download not in ('Preferred University', 'Evaluator', 'University Graduate',
                         'Marketing Lead', 'Existing Customer')
    and download not like '%Not Successful%'
)
select year, month, lead_type, count(lead_type) from tmp
    where lead_type in ('Event', 'Training', 'Collateral', 'TX-LINE', 'Webinar')
    group by year, month, lead_type;

""".format(begin, month_start)

conn = init_cdb()
results = defaultdict(OrderedDict)
cur = conn.cursor()
cur.execute(query)
for row in cur.fetchall():
    (year, month, lead_type, count) = row
    label = '{}-{:02}'.format(year, month)
    results[lead_type][label] = count

df = pd.DataFrame(results)

In [ ]:
plt.figure(1, figsize=(12, 12))
for sp, typ in enumerate(['Event', 'Training', 'Collateral', 'TX-LINE', 'Webinar']):
    plt.subplot(5, 1, sp+1)  # numrows, numcols, fignum
    plt.tight_layout()
    plt.title(typ)
    # changed to add x range when updated version of matplotlib
    plt.plot(np.arange(len(df)), df[typ], linewidth=2.0)
    plt.ylabel('Lead Count')
    xlabels = df[typ].index
    plt.xticks(range(len(xlabels)), xlabels, rotation=30)
plt.show()

# Backup Data

For completeness, here is the above data in tabular form.

In [ ]:
df